[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pdf-tools/components-code-sample-hub/blob/main/jupyter/pdftools_toolbox/pdftools_toolbox_add_metadata.ipynb)

In [ ]:
%pip install https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/productkits/PDFSDKXT/latest/pdftools_toolbox-latest.tar.gz
%pip install ipython

# Add metadata to PDF
Set metadata such as author, title, and creator of a PDF
document. Optionally use the metadata of another PDF
document or the content of an XMP file.

In [ ]:
import io
from pdftools_toolbox.pdf import Document, FileReference, Metadata, PageCopyOptions, PageList
from pdftools_toolbox.pdf.content import IccBasedColorSpace
from pdftools_toolbox.pdf.navigation import ViewerSettings

In [ ]:
# Download a file from a given URL and save it to the local system
def prepare_file(url: str, path: str):
    import requests
    response = requests.get(url)
    response.raise_for_status()

    with open(path, 'wb') as f:
        f.write(response.content)

In [ ]:
# Set input arguments
input_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/BlankNone.pdf'
input_file_path = 'BlankNone.pdf'
prepare_file(input_url, input_file_path)
metadata_url = 'https://pdftools-public-downloads-production.s3.eu-west-1.amazonaws.com/samples/testfiles/MetadataTest.xmp'
metadata_file_path = 'MetadataTest.xmp'
prepare_file(metadata_url, metadata_file_path)
output_file_path = 'BlankWithMetadata.pdf'

In [ ]:
def copy_document_data(in_doc: Document, out_doc: Document):
    # Copy document-wide data (excluding metadata)

    # Output intent
    if in_doc.output_intent is not None:
        in_doc.output_intent = IccBasedColorSpace.copy(out_doc, in_doc.output_intent)

    # Viewer settings
    out_doc.viewer_settings = ViewerSettings.copy(out_doc, in_doc.viewer_settings)

    # Associated files (for PDF/A-3 and PDF 2.0 only)
    outAssociatedFiles = out_doc.associated_files
    for in_file_ref in in_doc.associated_files:
        outAssociatedFiles.append(FileReference.copy(out_doc, in_file_ref))

    # Plain embedded files
    out_embedded_files = out_doc.plain_embedded_files
    for in_file_ref in in_doc.plain_embedded_files:
        out_embedded_files.append(FileReference.copy(out_doc, in_file_ref))

In [ ]:
try:
    # Set and check license key. If the license key is not valid, an exception is thrown.
    from pdftools_toolbox.sdk import Sdk
    Sdk.initialize("INSERT-LICENSE-KEY", None)

    # Open input document
    with io.FileIO(input_file_path, 'rb') as content_pdf_stream:
        with Document.open(content_pdf_stream, None) as content_pdf_document:
    
            # Create output document
            with io.FileIO(output_file_path, 'wb+') as output_stream:
                with Document.create(output_stream, content_pdf_document.conformance, None) as output_document:
                    # Copy document-wide data
                    copy_document_data(content_pdf_document, output_document)
    
                    # Set Metadata
                    if metadata_file_path is not None:
                        with io.FileIO(metadata_file_path, 'rb') as metadata_stream:
                            if metadata_file_path.endswith(".pdf"):
                                # Use the metadata of another PDF file
                                with Document.open(metadata_stream, "") as meta_doc:
                                    mdata = Metadata.copy(output_document, meta_doc.metadata)
                            else:
                                # Use the content of an XMP metadata file
                                mdata = Metadata.create(output_document, metadata_stream)
                    else:
                        mdata = output_document.metadata
                        mdata.author = "Your Author"
                        mdata.title = "Your Title"
                        mdata.subject = "Your Subject"
                        mdata.creator = "Your Creator"
    
                    output_document.metadata = mdata
    
                    # Define page copy options
                    copy_options = PageCopyOptions()
    
                    # Copy all pages and append to output document
                    copied_pages = PageList.copy(output_document, content_pdf_document.pages, copy_options)
                    output_document.pages.extend(copied_pages)

    print("Execution successful.")
except Exception as e:
    print(f"An error occurred: {e}")